In [114]:
import optuna
import subprocess
from multiprocessing import Pool
import os
import optuna
from random import sample
from tqdm import tqdm
from typing import List

def compile_cpp(cpp_file):
    exe_file = "a.out"
    # コンパイルする
    subprocess.run(["g++", "-O3", "-std=c++17", "-o", exe_file, cpp_file])
    return exe_file

# 標準出力をgeneratorとして返す
def execute_cpp_get_lines(cpp_file: str, args: List[str]):
    exe_file = compile_cpp(cpp_file)
    cmd = " ".join(["./"+exe_file, *args])
    # Popenは新規プロセスを立ち上げるので非同期的に処理する
    proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

    while True:
        line = proc.stdout.readline()
        if line:
            yield line

        if not line and proc.poll() is not None:
            break
            
# 結果だけを返す(コンパイルしないので実行ファイルのパスを指定する)
def execute_cpp(exe_file: str, args: List[str]):
    cmd = " ".join(["./"+exe_file, *args])
    # Popenは新規プロセスを立ち上げるので非同期的に処理する
    return subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

### ランダムに移動する

In [99]:
H = "3"
W = "4"
END_TURN = "4"
seed = "121321"

for line in execute_cpp_get_lines("onePlayerGame.cpp", [H, W, END_TURN, seed]):
    sys.stdout.write(line.decode('utf-8'))

turn:	0
score:	0
4613
.@2.
7566

turn:	1
score:	2
4613
..@.
7566

turn:	2
score:	3
46@3
....
7566

turn:	3
score:	3
46.3
..@.
7566

turn:	4
score:	3
46.3
...@
7566



### ビームサーチ

In [130]:
H = "5"
W = "5"
END_TURN = "5"
seed = "121321"
beam_width = "2"
beam_depth = END_TURN
is_tracking = "1" # 盤面を出力する場合に 1 

for line in execute_cpp_get_lines("beamSearch.cpp", [H, W, END_TURN, seed, beam_width, beam_depth, is_tracking]):
    sys.stdout.write(line.decode('utf-8'))

1
turn:	0
score:	0
4613.
2.@75
66663
41372
11652

turn:	1
score:	7
4613.
2..@5
66663
41372
11652

turn:	2
score:	13
4613.
2...5
666@3
41372
11652

turn:	3
score:	20
4613.
2...5
666.3
413@2
11652

turn:	4
score:	25
4613.
2...5
666.3
413.2
116@2

turn:	5
score:	31
4613.
2...5
666.3
413.2
11@.2

31


### ビームサーチの平均点を計算

In [135]:
from numpy.random import MT19937, Generator
H = "30"
W = "30"
END_TURN = "100"

beam_width = "5"
beam_depth = END_TURN
is_tracking = "0" # 盤面を出力する場合に 1 、そうでない場合は 0

# コンパイル
exe_file = compile_cpp("beamSearch.cpp")
# 乱数生成器
rg_mt = Generator(MT19937())

# ゲームを繰り返す回数
game_number = 100

score_sum = 0
for i in range(game_number):
    seed = str(int(str(rg_mt.random())[2:]))
    res = execute_cpp(exe_file, [H, W, END_TURN, seed, beam_width, beam_depth, is_tracking])
    game_score = int(res.stdout.decode('utf-8').replace('\n', ''))
    score_sum += game_score

print("{}回ゲームを繰り返した際の平均スコア {}".format(game_number, score_sum/game_number))

100回ゲームを繰り返した際の平均スコア 685.84
